In [0]:
import numpy as np 
import pandas as pd 
import io
import requests
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import itertools

In [0]:
class dlnet:
    def __init__(self, x, y):
        self.X=x
        self.Y=y
        self.Yh=np.zeros((1,self.Y.shape[1]))
        self.L=2
        self.dims = [8, 15, 1]
        self.param = {}
        self.ch = {}
        self.grad = {}
        self.loss = []
        self.lr=0.003
        self.sam = self.Y.shape[1]
        self.threshold=0.5
        
    def nInit(self):    
      np.random.seed(1)
      self.param['W1'] = np.random.randn(self.dims[1], self.dims[0]) / np.sqrt(self.dims[0]) 
      self.param['b1'] = np.zeros((self.dims[1], 1))        
      self.param['W2'] = np.random.randn(self.dims[2], self.dims[1]) / np.sqrt(self.dims[1]) 
      self.param['b2'] = np.zeros((self.dims[2], 1))                
      return
  
    def Sigmoid(self,Z):
      return 1/(1+np.exp(-Z))
  
    def Relu(self,Z):
        return np.maximum(0,Z)
      
    def forward(self):    
      
#         print("W1: {}".format(self.param['W1'].dot(self.X)))
#         print("B1: {}".format(self.param['b1']))
        
        Z1 = self.param['W1'].dot(self.X) + self.param['b1'] 
        
#         print("Z1 Values: {}".format(Z1))
        
        A1 = self.Relu(Z1)
        self.ch['Z1'],self.ch['A1']=Z1,A1

        Z2 = self.param['W2'].dot(A1) + self.param['b2']  
        A2 = self.Sigmoid(Z2)
        self.ch['Z2'],self.ch['A2']=Z2,A2
        self.Yh=A2
#         print(self.Yh)
        loss=self.nloss(A2)
        return self.Yh, loss
      
    def nloss(self,Yh):
      loss = (1/self.sam) * (-np.dot(self.Y,np.log(Yh).T) - np.dot(1-self.Y, np.log(1-Yh).T)) 
#       f = np.log(Yh).T
#       s = (np.dot(1-self.Y,np.log(Yh).T))
#       print(f)  
#       print(s)
      return loss
    
    def dRelu(self,x):
      x[x<=0] = 0
      x[x>0] = 1
      return x
  
    def dSigmoid(self,Z):
      s = 1/(1+np.exp(-Z))
      dZ = s * (1-s)
      return dZ
      
    def backward(self):
#       print("Yh: {}".format(self.Yh))
      
      dLoss_Yh = - (np.divide(self.Y, self.Yh ))
#       print("dLoss_Yh: {}".format(dLoss_Yh))
      
      dLoss_Z2 = dLoss_Yh * self.dSigmoid(self.ch['Z2'])    
#       print("dLoss_Z2: {}".format(dLoss_Z2))
      
      dLoss_A1 = np.dot(self.param["W2"].T,dLoss_Z2)
#       print("dLoss_A1: {}".format(dLoss_A1))
      
      dLoss_W2 = 1./self.ch['A1'].shape[1] * np.dot(dLoss_Z2,self.ch['A1'].T)
#       print("dLoss_W2: {}".format(dLoss_W2))
      
      dLoss_b2 = 1./self.ch['A1'].shape[1] * np.dot(dLoss_Z2, np.ones([dLoss_Z2.shape[1],1])) 
#       print("dLoss_b2: {}".format(dLoss_b2))

      dLoss_Z1 = dLoss_A1 * self.dRelu(self.ch['Z1'])      
#       print("dLoss_Z1: {}".format(dLoss_Z1))
      
      dLoss_A0 = np.dot(self.param["W1"].T,dLoss_Z1)
#       print("dLoss_A0: {}".format(dLoss_A0))
      
      dLoss_W1 = 1./self.X.shape[1] * np.dot(dLoss_Z1,self.X.T)
#       print("dLoss_W1: {}".format(dLoss_W1))
      
      dLoss_b1 = 1./self.X.shape[1] * np.dot(dLoss_Z1, np.ones([dLoss_Z1.shape[1],1])) 
#       print("dLoss_b1: {}".format(dLoss_b1))

      self.param["W1"] = self.param["W1"] - self.lr * dLoss_W1
      self.param["b1"] = self.param["b1"] - self.lr * dLoss_b1
      self.param["W2"] = self.param["W2"] - self.lr * dLoss_W2
      self.param["b2"] = self.param["b2"] - self.lr * dLoss_b2
        
    def gd(self,X, Y, iter = 3000):
      np.random.seed(1)                         

      self.nInit()

      for i in range(0, iter):
          Yh, loss=self.forward()
          self.backward()

          if i % 500 == 0:
              print ("Cost after iteration {}: {}" .format(i, loss))
#               print(loss)
              self.loss.append(loss)

      return
    
    def pred(self,x, y):  
        self.X=x
        self.Y=y
        comp = np.zeros((1,x.shape[1]))
        pred, loss= self.forward()    
    
        for i in range(0, pred.shape[1]):
            if pred[0,i] > 0.5: comp[0,i] = 1
            else: comp[0,i] = 0
    
        print("Acc: " + str(np.sum((comp == y)/x.shape[1])))
        
        return comp
      



In [0]:
url = "https://raw.githubusercontent.com/Nd98/Neural-Network-Dataset/master/Teaning%20Data%20set.csv"

df = pd.read_csv(url)

non_float_df = df

df.head(5)


,UTS,YS,C,Mn,Si,S,P,Cr,Ni,Mo
0,558.4,291.2,0.055,1.027,1.361,0.01,0.04,18.322,8.204,0.231
1,558.4,291.2,0.055,1.027,1.361,0.01,0.04,18.322,8.204,0.231
2,580.8,299.3,0.052,0.963,1.206,0.01,0.04,18.398,8.189,0.212
3,572.6,290.4,0.052,0.998,1.224,0.01,0.04,18.421,8.181,0.204
4,572.6,290.4,0.052,0.998,1.224,0.01,0.04,18.421,8.181,0.204


In [0]:
names = df.columns[0:11]
scaler = MinMaxScaler() 
# scaler
scaled_df = scaler.fit_transform(df.iloc[:,0:11]) 
scaled_df = pd.DataFrame(scaled_df, columns=names)

# scaled_df['UTS'] = non_float_df['UTS']
# scaled_df['YS'] = non_float_df['YS']
scaled_df['S'] = non_float_df['S']

scaled_df.head(5)


,UTS,YS,C,Mn,Si,S,P,Cr,Ni,Mo
0,0.635480,0.835697,0.448276,0.538012,0.979675,0.01,1.0,0.582205,0.434004,0.589109
1,0.635480,0.835697,0.448276,0.538012,0.979675,0.01,1.0,0.582205,0.434004,0.589109
2,0.907655,0.931442,0.344828,0.413255,0.349593,0.01,1.0,0.729207,0.400447,0.495050
3,0.808019,0.826241,0.344828,0.481481,0.422764,0.01,1.0,0.773694,0.382550,0.455446
4,0.808019,0.826241,0.344828,0.481481,0.422764,0.01,1.0,0.773694,0.382550,0.455446


In [0]:
x=scaled_df.iloc[0:280,2:10].values.transpose()
y=df.iloc[0:280,0:1].values.transpose()
xval=scaled_df.iloc[280:359,2:10].values.transpose()
yval=df.iloc[280:359,0:1].values.transpose()

In [0]:
nn = dlnet(x,y)
nn.lr=0.001

# print(np.zeros((1,y.shape[1])))
nn.dims = [8, 15, 1]
nn.gd(x, y, iter = 10000)

Cost after iteration 0: [[12.11931521]]
Cost after iteration 500: [[-5045.75463213]]
Cost after iteration 1000: [[-5530.18833556]]
Cost after iteration 1500: [[-5813.00807826]]
Cost after iteration 2000: [[-6013.6455773]]
Cost after iteration 2500: [[-6169.33190454]]
Cost after iteration 3000: [[-6296.59905259]]
Cost after iteration 3500: [[-6404.26083388]]
Cost after iteration 4000: [[-6497.58359357]]
Cost after iteration 4500: [[-6579.93908349]]
Cost after iteration 5000: [[-6653.64255087]]
Cost after iteration 5500: [[-6720.34642111]]
Cost after iteration 6000: [[-6781.2741356]]
Cost after iteration 6500: [[-6837.34737433]]
Cost after iteration 7000: [[-6889.28304273]]
Cost after iteration 7500: [[-6937.65206076]]
Cost after iteration 8000: [[-6982.91446046]]
Cost after iteration 8500: [[-7025.44660353]]
Cost after iteration 9000: [[-7065.56304156]]
Cost after iteration 9500: [[-7103.52327124]]
